Librerias necesarias

In [13]:
# !pip install pandas numpy joblib xgboost

# Importaciones
import pandas as pd
import numpy as np
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

Carga de modelos

In [14]:
# Definición de características (orden crucial)
features_ush = ['anio', 'temp_max', 'temp_min', 'temp_media', 'lluvia_mm', 'dias_nieve',
                'ent_san_sebastian', 'aero_ush', 'aero_rg',
                'mes_abril', 'mes_agosto', 'mes_diciembre', 'mes_enero', 'mes_febrero',
                'mes_julio', 'mes_junio', 'mes_marzo', 'mes_mayo', 'mes_noviembre',
                'mes_octubre', 'mes_septiembre']

features_rg = [f for f in features_ush if f != 'mes_julio']# Río Grande excluye 'mes_julio'
trained_models = {}
model_paths = {
    'ush_toh': 'best_model_ush_toh.joblib', #Modifica cada uno para tu ruta
    'ush_top': 'best_model_ush_top.joblib', #por ejemplo 'ush_toh': '/content/drive/MyDrive/MiCarpetaDeModelos/best_model_ush_toh.joblib',
    'rg_toh_no_julio': 'best_model_rg_toh_no_julio.joblib',
    'rg_top_no_julio': 'best_model_rg_top_no_julio.joblib'
}

for key, path in model_paths.items():
    if os.path.exists(path):
        try:
            trained_models[key] = joblib.load(path)
            print(f"  Modelo '{key}' cargado.")
        except Exception as e:
            print(f"  Error cargando '{key}': {e}")
    else:
        print(f"  Advertencia: Archivo '{path}' no encontrado para '{key}'.")
print("Carga de modelos completada.")

  Modelo 'ush_toh' cargado.
  Modelo 'ush_top' cargado.
  Modelo 'rg_toh_no_julio' cargado.
  Modelo 'rg_top_no_julio' cargado.
Carga de modelos completada.


Función amigable para predecir

In [15]:
def make_prediction(model_key, new_data_df):
    """Realiza una predicción usando un modelo pre-entrenado."""
    if model_key not in trained_models:
        print(f"Error: Modelo '{model_key}' no disponible.")
        return None

    model = trained_models[model_key]
    expected_features = features_rg if 'rg' in model_key else features_ush

    processed_data = pd.DataFrame(0, index=new_data_df.index, columns=expected_features)
    for col in expected_features:
        if col in new_data_df.columns:
            # Convierte 'mes_' booleanos a int (0/1).
            processed_data[col] = new_data_df[col].astype(int) if col.startswith('mes_') and new_data_df[col].dtype == 'bool' else new_data_df[col]
    processed_data = processed_data[expected_features] # Asegura el orden.

    return model.predict(processed_data)


#Ejemplo de uso

In [16]:
# Datos para Ushuaia (Diciembre 2022)
ush_data_diciembre_2022 = {
    'anio': [2022], 'temp_max': [14.5], 'temp_min': [5.2], 'temp_media': [9.5],
    'lluvia_mm': [58.2], 'dias_nieve': [0], 'ent_san_sebastian': [17719],
    'aero_ush': [54968.0], 'aero_rg': [5630],
    'mes_abril': [False], 'mes_agosto': [False], 'mes_diciembre': [True], 'mes_enero': [False],
    'mes_febrero': [False], 'mes_julio': [False], 'mes_junio': [False], 'mes_marzo': [False],
    'mes_mayo': [False], 'mes_noviembre': [False], 'mes_octubre': [False], 'mes_septiembre': [False]
}
ush_df = pd.DataFrame(ush_data_diciembre_2022)

pred_ush_toh = make_prediction('ush_toh', ush_df)
if pred_ush_toh is not None: print(f"Predicción TOH% Ushuaia (Dic 2022): {pred_ush_toh[0]:.2f}%")
pred_ush_top = make_prediction('ush_top', ush_df)
if pred_ush_top is not None: print(f"Predicción TOP% Ushuaia (Dic 2022): {pred_ush_top[0]:.2f}%")

# Datos para Río Grande (Diciembre 2022)
rg_data_diciembre_2022 = {
    'anio': [2022], 'temp_max': [15.9], 'temp_min': [4.5], 'temp_media': [10.3],
    'lluvia_mm': [17.5], 'dias_nieve': [0], 'ent_san_sebastian': [17719.0],
    'aero_ush': [54968.0], 'aero_rg': [5630],
    'mes_abril': [False], 'mes_agosto': [False], 'mes_diciembre': [True], 'mes_enero': [False],
    'mes_febrero': [False], 'mes_julio': [False], # 'mes_julio' será ignorado para RG
    'mes_junio': [False], 'mes_marzo': [False], 'mes_mayo': [False], 'mes_noviembre': [False],
    'mes_octubre': [False], 'mes_septiembre': [False]
}
rg_df = pd.DataFrame(rg_data_diciembre_2022)

pred_rg_toh = make_prediction('rg_toh_no_julio', rg_df)
if pred_rg_toh is not None: print(f"Predicción TOH% Río Grande (Dic 2022, Julio en input, ignorado): {pred_rg_toh[0]:.2f}%")
pred_rg_top = make_prediction('rg_top_no_julio', rg_df)
if pred_rg_top is not None: print(f"Predicción TOP% Río Grande (Dic 2022, Julio en input, ignorado): {pred_rg_top[0]:.2f}%")

Predicción TOH% Ushuaia (Dic 2022): 60.61%
Predicción TOP% Ushuaia (Dic 2022): 45.67%
Predicción TOH% Río Grande (Dic 2022, Julio en input, ignorado): 28.21%
Predicción TOP% Río Grande (Dic 2022, Julio en input, ignorado): 17.50%


###Para Ushuaia (Diciembre 2022):

Valores Reales (de tus datos de entrada):

TOH%: 59.7%  
TOP%: 45.3%  
  
Tus Predicciones:

Predicción TOH% Ushuaia: 60.93%  
Predicción TOP% Ushuaia: 45.54%  
   
###Para Río Grande (Diciembre 2022):

Valores Reales (de tus datos de entrada):

TOH%: 28.2%  
TOP%: 16.0%  
  
Tus Predicciones:  
Predicción TOH% Río Grande: 28.00%    
Predicción TOP% Río Grande: 17.28%  